In [114]:
%run "../layeringOfServices.ipynb"

In [115]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
from datetime import datetime

In [116]:
from re import sub
def name_handler(s):
    s = sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespace with a single dash
    s = sub(r"\s+", '_', s)
    return s



In [117]:
tested_hts = dreams_mastersheet[(dreams_mastersheet.timeOn_system == "required_Time_on")&(dreams_mastersheet.hts == "tested_on_given_date")&(dreams_mastersheet.newage_range != "10-17")&(dreams_mastersheet.newage_range != "25-29")&(dreams_mastersheet.newage_range != "not_valid_age")]

In [118]:
tested_hts_21 = tested_hts[tested_hts.fiscal_year != "not_valid_fy"]

## Nouvelles Filles HTS

In [119]:
hts_new = dreams_mastersheet[(dreams_mastersheet.timeOn_system == "required_Time_on")&(dreams_mastersheet.hts == "errata")&(dreams_mastersheet.newage_range != "10-17")&(dreams_mastersheet.newage_range != "25-29")&(dreams_mastersheet.newage_range != "not_valid_age")]

In [120]:
date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [121]:
hts_new.to_excel(f"verification_{date_du_jour}.xlsx",index=False,na_rep="NULL")

## Oldest

> verification

Pick the verification as the starting point

In [122]:
old_hts = pd.read_excel("verification.xlsx")

In [123]:
old_hts.code.count()
hts_new.code.count()
tested_hts.code.count()
tested_hts_21.code.count()

4799

4930

577

34

In [124]:
hts_new.date_entevyou.min().strftime("%Y-%m-%d")
hts_new.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-07'

'2021-06-23'

In [125]:
old_hts.date_entevyou.min().strftime("%Y-%m-%d")
old_hts.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-07'

'2021-06-15'

In [126]:
tested_hts.date_entevyou.min().strftime("%Y-%m-%d")
tested_hts.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-18'

'2021-06-05'

In [127]:
tested_hts_21.date_entevyou.min().strftime("%Y-%m-%d")
tested_hts_21.date_entevyou.max().strftime("%Y-%m-%d")

'2020-10-05'

'2021-06-05'

In [128]:
howmuch_htsNew_isin_htsold = hts_new[hts_new.code.isin(old_hts.code)]

In [129]:
howmuch_htsNew_isnot_htsold = hts_new[~hts_new.code.isin(old_hts.code)]

In [130]:
howmuch_tested_isin_htsold = tested_hts[tested_hts.code.isin(old_hts.code)]

In [131]:
howmuch_tested21_isin_htsold =  tested_hts_21[tested_hts_21.code.isin(old_hts.code)]

In [132]:
howmuch_htsNew_isin_htsold.code.count()
howmuch_htsNew_isnot_htsold.code.count()
howmuch_tested_isin_htsold.code.count()
howmuch_tested21_isin_htsold.code.count()

4776

154

9

7

In [133]:
howmuch_htsNew_isin_htsold.date_entevyou.min().strftime("%Y-%m-%d")
howmuch_htsNew_isin_htsold.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-07'

'2021-06-15'

In [134]:
howmuch_htsNew_isnot_htsold.date_entevyou.min().strftime("%Y-%m-%d")
howmuch_htsNew_isnot_htsold.date_entevyou.max().strftime("%Y-%m-%d")

'2020-05-29'

'2021-06-23'

In [135]:
howmuch_tested_isin_htsold.date_entevyou.min().strftime("%Y-%m-%d")
howmuch_tested_isin_htsold.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-18'

'2021-03-08'

In [136]:
howmuch_tested21_isin_htsold.date_entevyou.min().strftime("%Y-%m-%d")
howmuch_tested21_isin_htsold.date_entevyou.max().strftime("%Y-%m-%d")

'2020-11-10'

'2021-03-08'

## Report Component

In [137]:
hts_new_communes = []
for columns in hts_new.actual_commune.unique():
    globals()[f"hts_new_{name_handler(columns)}"] = hts_new[hts_new.actual_commune == columns]
    hts_new_communes.append(globals().get(f"hts_new_{name_handler(columns)}"))

In [138]:
new_communes_general = hts_new.actual_commune.unique().tolist()
hts_new_dict = {}
for columns, communes in zip(hts_new.actual_commune.unique(), new_communes_general):
    hts_new_dict[f"{name_handler(communes)}"] =  globals().get(f"hts_new_{name_handler(columns)}").code.count()
    
    
hts_new_df = pd.DataFrame.from_dict(hts_new_dict, orient="index", columns=None)
hts_new_df.columns = ["nombre_filles_pour_depistage_hts"]
hts_new_df

,nombre_filles_pour_depistage_hts
Dessalines,171
Kenscoff,780
Gressier,271
CapHaïtien,237
Milot,76
PlaineduNord,89
QuartierMorin,37
Limonade,116
Verrettes,543
Liancourt,29


In [139]:
liste_hts_new = pd.ExcelWriter(f"rapportage_liste_filles_hts_{date_du_jour}.xlsx",engine="xlsxwriter")
hts_new_df.to_excel(liste_hts_new,sheet_name="Total_filles_pr_commune",index=True)
hts_new.to_excel(liste_hts_new,sheet_name="liste_des_filles_global",index=False,na_rep="NULL")
for columns,communes in zip(hts_new.actual_commune.unique(), new_communes_general):
    globals().get(f"hts_new_{name_handler(columns)}").to_excel(liste_hts_new,sheet_name=f"{name_handler(communes)}",index=False,na_rep="NULL")
liste_hts_new.save()